# Scripts for volatile additions to yeast

In [2]:
import os
import ioyeast as io

In [5]:
# os.chdir(os.path.abspath(os.pardir))
# os.getcwd()
MODEL_PATH = os.path.join(os.path.abspath(os.pardir), 'model','yeast-GEM.xml')
model = io.read_yeast_model(MODEL_PATH)
model

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


Name,yeastGEM_develop
Memory address,0x07f9acefa1fd0
Number of metabolites,2744
Number of reactions,4063
Number of groups,106
Objective expression,1.0*r_2111 - 1.0*r_2111_reverse_58b69
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"
